# Generador de actualizaciones sql en python

In [3]:
from tqdm import tqdm
import pandas as pd
pd.options.display.max_columns = None
from datetime import datetime, timedelta

Lectura de datos

In [5]:
df = pd.read_excel(r'Updates\Actualizaciones.xlsx')

## Cotrol de datos

In [6]:
# Cambiar la fecha y hora sumando un offset de tiempo
df['Fecha'] = df['Fecha'] + pd.DateOffset(days=3)
df

,FacturaId,Fecha
0,54719,2023-07-03 09:14:00
1,54720,2023-07-03 09:17:00
2,54721,2023-07-03 09:19:00
3,54722,2023-07-03 09:21:00
4,54723,2023-07-03 09:24:00
...,...,...
121,54840,2023-07-03 15:16:00
122,54841,2023-07-03 15:19:00
123,54842,2023-07-03 15:22:00
124,54843,2023-07-03 15:23:00


In [8]:
hora_objetivo_inicio = datetime.strptime('07:00:00', '%H:%M:%S').time()
hora_objetivo_fin = datetime.strptime('09:00:00', '%H:%M:%S').time()

In [9]:
def generar_fechas():
    fecha_actual = datetime.combine(datetime.strptime('2023-07-03', '%Y-%m-%d').date(), hora_objetivo_inicio)

    fechas = []
    for _ in range(126):
        fechas.append(fecha_actual)
        fecha_actual += timedelta(minutes=120/126)

    return fechas

# Obtener las fechas generadas
fechas_generadas = generar_fechas()

# Agregar las fechas generadas a la columna 'fecha' del DataFrame
df['Fecha'] = fechas_generadas

In [11]:
df.head(3)

,FacturaId,Fecha
0,54719,2023-07-03 07:00:00.000000
1,54720,2023-07-03 07:00:57.142857
2,54721,2023-07-03 07:01:54.285714


In [13]:
df['fecha_formateada'] = df['Fecha'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
df.head(3)

,FacturaId,Fecha,fecha_formateada
0,54719,2023-07-03 07:00:00.000000,2023-07-03 07:00:00
1,54720,2023-07-03 07:00:57.142857,2023-07-03 07:00:57
2,54721,2023-07-03 07:01:54.285714,2023-07-03 07:01:54


# Crear Query

In [116]:
def query (df:pd.DataFrame, fileName:str, tableBDD:str, columnUD:str, columnDF:str, WhereBDD:str, WhereDF:str)->None:    
    lista = []
    a = """
DECLARE
    @ErrorMessage NVARCHAR(4000),
    @ErrorSeverity INT,
    @ErrorState INT"""
   
    lista.append(a)
    for index, row in df.iterrows():
        a = f"""
BEGIN TRY
    BEGIN TRANSACTION
        UPDATE {tableBDD}
        SET {columnUD} = '{row[f'{columnDF}']}'
        WHERE {WhereBDD} = {row[f'{WhereDF}']}
    COMMIT TRANSACTION;
END TRY
BEGIN CATCH
    SELECT 
        @ErrorMessage = ERROR_MESSAGE(),
        @ErrorSeverity = ERROR_SEVERITY(),
        @ErrorState = ERROR_STATE();
    RAISERROR (@ErrorMessage, @ErrorSeverity, @ErrorState);
    ROLLBACK TRANSACTION;
END CATCH"""
        lista.append(a)

    nombre_archivo = f'{fileName}.sql'
    with open(nombre_archivo, 'w') as archivo:
        for sublist in lista:
            fila = ''.join(map(str, sublist))
            archivo.write(fila + '\n')

ejecucion

In [117]:
# query (df, fileName, tableBDD, columnUD, columnDF, WhereBDD, WhereDF)
query(df,'prueba','Sivev.Facturas.Facturas','Fecha','fecha_formateada','FacturaId','FacturaId')